In [2]:
import numpy as np
from scipy.stats import binom
import pandas as pd
from itertools import combinations

carga = 2850

In [3]:
class Agrupamento:
    def __init__(self, pot, qtd, prob):
        self.pot = pot
        self.qtd = qtd
        self.prob = prob
        self.geraTabela()

    def geraTabela(self):
        self.resultados = []
        self.iterado = []
        for i in range(int(self.qtd) + 1):
            self.resultados.append((i * self.pot, binom.pmf(i, int(self.qtd), self.prob)))
            self.iterado.append(0)
        self.usina = pd.DataFrame(self.resultados,columns=['Pot','Prob'])
        self.usina['Iter'] = self.iterado
        self.usina['IterAg'] = np.nan
          


def Calcula_COPT(data, carga):
    agrupamento = []
    usinas = []
    geracao_total = 0
    for i in range(len(data['Usina'])):
        agrupamento.append(Agrupamento(data.loc[i]['Pot. Ativa Max'], data.loc[i]['Unidades'], data.loc[i]['FOR']))
        usinas.append(agrupamento[i].usina)
        geracao_total += data.loc[i]['Unidades']*data.loc[i]['Pot. Ativa Max']
    
    potencia_para_falta = geracao_total - carga
    
    lista = []
    for i in range(len(usinas)):
        for x in range(len(usinas[i])):
            lista.append((i,x))
    
    combinacoes = []
    for i in combinations(lista,2):
        combinacoes.append(i)
        
    
    
    resultados_final = []
    
    for i in range(len(combinacoes)):
        soma = 0
        produto = 1
        for j in combinacoes[i]:
            soma += agrupamento[j[0]].resultados[j[1]][0]
            produto *= agrupamento[j[0]].resultados[j[1]][1]
        resultados_final.append((soma,produto))
    
    
    
    copt = pd.DataFrame(resultados_final, columns=['Pot indisponivel', 'Probabilidade'])
    copt = copt.groupby(by = ['Pot indisponivel'], as_index = False).sum()
    
    perda_de_carga = []
    
    for i in range(len(copt)):
        if copt.loc[i]['Pot indisponivel'] <=  potencia_para_falta:
            perda_de_carga.append(0)
        else:
            perda_de_carga.append(abs(copt.loc[i]['Pot indisponivel'] - potencia_para_falta))
    
    
    copt['Perda de Carga'] = perda_de_carga
    copt['x.p'] = copt['Perda de Carga'] * copt['Probabilidade']
    
    EPNS = sum(copt['x.p'])
    LOLP = sum(copt[copt['Perda de Carga'] > 0]['Probabilidade'])
    LOLE = LOLP * 8760
    EENS = EPNS * 8760
    resul_copt = [EPNS, EENS, LOLP, LOLE]
    return resul_copt



def pu(x):
    return x/100

In [4]:
data = pd.read_excel('Gerac.xlsx')
data['FOR'] = data['FOR'].apply(pu)

In [5]:
dados_carga = pd.read_excel('dataframe_curva_de_carga.xlsx')
dados_carga

,Num.,Valor de Carga,Probabilidade,Prob. Acumulada
0,0,96,0.069406,1.000000
1,1,100,0.066324,0.930594
2,2,90,0.060502,0.864269
3,3,95,0.060388,0.803767
4,4,92,0.059817,0.743379
5,5,99,0.053653,0.683562
6,6,93,0.046119,0.629909
7,7,60,0.039612,0.583790
8,8,63,0.032192,0.544178
9,9,59,0.032192,0.511986


In [6]:
EPNS = []
EENS = []
LOLP = []
LOLE = []
valores_de_carga = dados_carga['Valor de Carga'].values

for n,i in enumerate(valores_de_carga):
    copt = Calcula_COPT(data, carga*i/100)
    EPNS.append(copt[0]*dados_carga['Probabilidade'][n])
    EENS.append(copt[1]*dados_carga['Probabilidade'][n])
    LOLP.append(copt[2]*dados_carga['Probabilidade'][n])
    LOLE.append(copt[3]*dados_carga['Probabilidade'][n])
    


In [13]:
E_sob_curva = sum(dados_carga['Valor de Carga'] * dados_carga['Probabilidade'] * 8760) * carga

EIR = 1 - sum(EENS)/E_sob_curva

0.9999928893053863

In [15]:
print('A LOLP do problema é: {}'.format(sum(LOLP)))
print('A EPNS do problema é: {} MW'.format(sum(EPNS)))
print('A LOLE do problema é: {} horas'.format(sum(LOLE)))
print('A EENS do problema é: {} MWh'.format(sum(EENS)))
print('A EIR  do problema é: {}'.format(EIR))

A LOLP do problema é: 0.01700738900079582
A EPNS do problema é: 1.6766214293238975 MW
A LOLE do problema é: 148.98472764697138 horas
A EENS do problema é: 14687.203720877344 MWh
A EIR  do problema é: 0.9999928893053863
